In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as r
import pickle as pkl
from types import SimpleNamespace
import os
import time

In [2]:
cwd = os.getcwd()
cwd

'd:\\D_Drive\\Github\\Thesis\\GraphSVX_graph_properties\\graph_theoretic_properties'

In [3]:
data = SimpleNamespace()
with open('../data/BA-2motif.pkl', 'rb') as fin:
    data.edge_index, data.x, data.y = pkl.load(fin)

In [17]:
data.indices = np.array(range(data.edge_index.shape[0]))
data.indices = np.reshape(data.indices,(data.indices.shape[0],-1))      # Important step
data.indices.shape

(1000, 1)

In [30]:
# For the duplication
new_data = SimpleNamespace()
new_data.indices = np.repeat(data.indices, 2)
new_data.edge_index = []
new_data.x = []
new_data.y = []

for i in list(range(data.edge_index.shape[0])):
    new_data.edge_index.append(data.edge_index[i])
    new_data.x.append(data.x[i])
    new_data.y.append(data.y[i])

    new_data.edge_index.append(data.edge_index[i])
    new_data.x.append(data.x[i])
    new_data.y.append(data.y[i])

new_data.edge_index = np.array(new_data.edge_index)
new_data.x = np.array(new_data.x)
new_data.y = np.array(new_data.y)

# Reshape the indices array
new_data.indices = np.reshape(new_data.indices, (new_data.indices.shape[0],-1))

# Save the dataset
start = time.time()

with open('../data/syn6_identical.pkl', 'wb') as f1:
    pkl.dump([new_data.indices, new_data.edge_index, new_data.x, new_data.y], f1)

print('Time (sec):',time.time() - start)

Time (sec): 0.015526056289672852


In [31]:
len(new_data.edge_index)

2000

In [9]:
a = np.random.randint(3, size=(4,4))
a = (a != 0)*1
a

array([[1, 1, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0]])

In [52]:
import math
import random
def decode(i):
    k = math.floor((1+math.sqrt(1+8*i))/2)  # eq1
    return k,i-k*(k-1)//2

def rand_pair(n):
    return decode(random.randrange(n*(n-1)//2))

def rand_pairs(n,m):
    return [decode(i) for i in random.sample(range(n*(n-1)//2),m)]

In [59]:
def pert_matrix(arr):
    """Perturbs 10% of the values in the matrix (excludes diagonal elements)

    Args:
        arr (numpy ndarray): matrix to be perturbed

    Returns:
        numpy ndarray: perturbed matrix
    """
    num_val_perturb = int(0.1*arr.shape[0]*arr.shape[1])

    # rand_nums1 = random.sample(range(size[0]),num_val_perturb)
    # rand_nums2 = random.sample(range(size[1]),num_val_perturb)
    rand_nos = rand_pairs(min(arr.shape[0],arr.shape[1]), num_val_perturb)

    for i in range(num_val_perturb):
        p = rand_nos[i][0]
        q = rand_nos[i][1]
        # Note that p==q never happends due to the decode method
        # If k = i - k(k-1)//2 then k won't satisfy the eq1
        
        arr[p][q] = 1.0 - arr[p][q]
    
    return arr
        
        

In [60]:
pert_data = SimpleNamespace()
pert_data.indices = new_data.indices
pert_data.edge_index = new_data.edge_index
pert_data.x = new_data.x
pert_data.y = new_data.y
for i in list(range(1,len(pert_data.indices),2)):
    # Perturbing the adjacency matrix
    a = pert_matrix(pert_data.edge_index[i])
    np.fill_diagonal(a,0)
    pert_data.edge_index[i] = a

    # Perturbing the feature matrix
    b = pert_matrix(pert_data.x[i])
    np.fill_diagonal(b,0)
    pert_data.x[i] = b

# flag = -1
# for i in range(data.y.shape[0]):
#     if np.array_equal(pert_data.y[2*i], pert_data.y[(2*i) + 1]) == False:
#         print(i)
#         flag = 0

# flag
print(pert_data.indices.shape)
print(pert_data.edge_index.shape)
print(pert_data.x.shape)
print(pert_data.y.shape)

(2000, 1)
(2000, 25, 25)
(2000, 25, 10)
(2000, 2)


In [61]:
start = time.time()

with open('../data/syn6_similar.pkl', 'wb') as f1:
    pkl.dump([new_data.indices, new_data.edge_index, new_data.x, new_data.y], f1)

print('Time (sec):',time.time() - start)

Time (sec): 0.018721342086791992
